In [324]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [325]:
import numpy as np 
import pandas as pd
import seaborn as sns
import argparse
import sklearn.utils.random as sk_random

## Introducción al dataset

El dataset contiene 5531451 filas, cada una correspondiente a la información de un cliente de la empresa American Express con 190 columnas que brindan información de los clientes. Por motivos de privacidad el significado de los valores de dichas columnas no es brindado por la empresa. Sin embargo en lugar del significado de cada columna se nos proporciona las siguientes categorías según el nombre de las mismas:
- D_* = Variables relacionadas a la delincuencia
- S_* = Variables relacionadas a los gastos
- P_* = Variables relacionadas a los pagos
- B_* = Variables de balances
- R_* = Variables de riesgo

Por último se aclara que todas las variables numéricas están normalizadas y que las siguientes variables son categóricas:

- ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']


## Reducción del dataset

Debido a la gran cantidad de información del dataset y por lo tanto peso del mismo, se redujo la cantidad de filas en un 5%. De esta forma y redondeando el resultado se plantea trabajar con un dataset que contenga unas 276572 filas.
El procedimiento utilizado para la reducción del dataset es el provisto por la función sample_without_replacement de la libreria scikit-learn. Como parámetros de la misma se utilizó:
- La cantidad de filas totales del dataset(n_population) = 5531451
- La cantidad de filas que se desean obtener(n_samples) = 276572
- Valor semilla (random_state) = 576

*Aclaración: Este último valor se calculó como el resto obtenido al realizar la siguiente división con la multiplicación de una constante por el número del grupo (11) como numerador y a 1000 como denominador $\frac{31416 * 11}{1000}$*

## a) Visualización de los datos:

In [326]:
df_reducido = pd.read_parquet('/kaggle/input/train-data/result_2')

In [327]:
df_reducido.shape

In [328]:
df_reducido

In [329]:
df_reducido['target'].value_counts()

### Tipos de variables

In [330]:
df_reducido.dtypes.value_counts()

Analizamos el tipo de las variables para conocer más de las mismas y determinar si serán de utilidad, o en su defecto si se debe realizar algún procedimiento para su modificación o eliminación.
Sabemos que las variables normalizadas pertenecen a la categoría “float64” y resulta de interés el análisis de las variables de tipo “object” u objetos y las variables “int” o enteras. 
- Las variables de tipo objeto son las columnas “customer_ID”,”S_2”,”D_63” y “D_64”
- Las variables de tipo entero son las columnas “target” y “B_31”

Se mostraran sus valores y se decidirá en base a los mismos que acciones tomar al momento de modificar el dataset.

In [331]:
df_reducido.loc[:,'S_2'] 

In [332]:
df_reducido['S_2'].isna().sum()

In [333]:
df_reducido.loc[:,'B_31']

In [334]:
df_reducido['B_31'].isna().sum()

In [335]:
df_reducido.loc[:,'D_63']

In [336]:
df_reducido['D_63'].isna().sum()

In [337]:
df_reducido.loc[:,'D_64']

In [338]:
df_reducido['D_64'].isna().sum()

### Categorias de las variables

In [339]:
columns = df_reducido.columns

sum_types = [0,0,0,0,0] # [type_D ,type_S,type_P,type_B, type_R ]

for column in columns:
    if (column[0] == 'D'):
        sum_types[0] += 1
    elif (column[0] == 'S'):
        sum_types[1] += 1
    elif (column[0] == 'P'):
        sum_types[2] += 1
    elif (column[0] == 'B'):
        sum_types[3] += 1
    elif (column[0] == 'R'):
        sum_types[4] += 1
sum_types

Observamos que en nuestro dataset reducido tenemos cantidades significativamente diferentes de cada tipo de variable, siendo 96 variables relacionadas a la delincuencia, 22 relacionadas a los gastos, 3 relacionadas a los pagos, 40 de balances y 28 variables de riesgo. Sumando un total 189 columnas donde se excluyen a dos variables ('customer_ID' y 'target') para completar las 191 variables tal y como se mostró anteriormente al realizar shape sobre el dataset.

In [340]:
df_reducido.sort_index(axis = 1, inplace = True)
df_reducido.reset_index(drop = True, inplace = True)
df_reducido

### Correlación entre variables

In [341]:
sns.heatmap(df_reducido.corr(method='pearson'), cmap='Reds')

Del heatmap obtenido se pueden observar ciertas correlaciones, siendo muchas de estas con un color intenso (indicando un gran nivel de correlación) con variables de su mismo “tipo” según las categorías presentadas al inicio del trabajo (D, S, P, B, R). Esto nos indica que existe la posibilidad de realizar una reducción de la dimensionalidad de los datos sin perder información, lo cual será profundizado más adelante en el trabajo.
Paralelamente observamos que existe una “Cruz” creada por una línea vertical y una horizontal de color blanco indicando que la variable D_87 no posee ninguna correlación con ninguna otra variable. Esta situación es causada por la cantidad de datos Nan que posee esta columna (mostrado a continuación), más adelante en el trabajo se trabajara sobre los datos faltantes y se tomaran acciones sobre los mismos.


In [342]:
df_reducido.loc[:,'D_87']

### Verificación de registros repetidos



Mediante la siguiente línea se verificamos que el dataset no posea filas filas con información repetida.

In [343]:
df_reducido.duplicated().value_counts()

### Relacion de variables con el target

In [344]:
import matplotlib.pyplot as plt

In [345]:
df_reducido['D_63'].value_counts()

In [346]:
deuda_D_63 = df_reducido.loc[:, ['target', 'D_63']].groupby(['D_63'])['target'].value_counts().unstack()
deuda_D_63

In [347]:
plt.figure(figsize=(12,12))
plt.xlabel('Deuda', size = 14)
plt.ylabel('D_63', size = 14)
plt.title('Deuda segun D_63', size = 18)
sns.heatmap(deuda_D_63, annot=True, fmt=".1f", linewidths=.5, square = True, cmap = 'Reds')

In [348]:
df_reducido['B_31'].value_counts()

In [349]:
df_reducido['D_64'].value_counts()

In [350]:
deuda_D_64 = df_reducido.loc[:, ['target', 'D_64']].groupby(['D_64'])['target'].value_counts().unstack()
deuda_D_64

In [351]:
plt.figure(figsize=(12,12))
plt.xlabel('Deuda', size = 14)
plt.ylabel('D_64', size = 14)
plt.title('Deuda segun D_64', size = 18)
sns.heatmap(deuda_D_64, annot=True, fmt=".1f", linewidths=.5, square = True, cmap = 'Reds')

Análisis de la variable "S_2"; la idea es ver si la fecha de cada transacción se relaciona de alguna manera con el objetivo, y si aporta información relevante. De no ser así, se podrá eliminar esta columna.

In [352]:
df_reducido['S_2'] = pd.to_datetime(df_reducido['S_2'], format='%Y-%m-%d')
df_reducido['S_2']

In [353]:
df_reducido['year'] = pd.DatetimeIndex(df_reducido['S_2']).year
df_reducido['year']

In [354]:
df_reducido['year'].value_counts()

In [355]:
year_target = df_reducido.groupby(['year'])['target'].sum().reset_index()
year_target

In [356]:
g = sns.barplot(x=year_target.year, y=year_target.target, palette='husl')
g.set_title("Incumplimientos de pago por año", fontsize=15)
g.set_xlabel("Año", fontsize=12)
g.set_ylabel("Número de pagos incumplidos", fontsize=12)

In [357]:
year_2017 = df_reducido.loc[df_reducido['year'] == 2017, ['S_2', 'target']]
year_2017

In [358]:
year_2017['month'] = pd.DatetimeIndex(year_2017['S_2']).month
year_2017['month'].value_counts()

In [359]:
months_2017 = year_2017.groupby(['month'])['target'].sum().reset_index()
months_2017

In [360]:
g = sns.barplot(x=months_2017.month, y=months_2017.target, palette='husl')
g.set_title("Incumplimientos de pago por mes en 2017", fontsize=15)
g.set_xlabel("Mes", fontsize=12)
g.set_ylabel("Número de pagos incumplidos", fontsize=12)

In [361]:
year_2018 = df_reducido.loc[df_reducido['year'] == 2018, ['S_2', 'target']]
year_2018

In [362]:
year_2018['month'] = pd.DatetimeIndex(year_2018['S_2']).month
year_2018['month'].value_counts()

In [363]:
months_2018 = year_2018.groupby(['month'])['target'].sum().reset_index()
months_2018

In [364]:
g = sns.barplot(x=months_2018.month, y=months_2018.target, palette='husl')
g.set_title("Incumplimientos de pago por mes en 2018", fontsize=15)
g.set_xlabel("Mes", fontsize=12)
g.set_ylabel("Número de pagos incumplidos", fontsize=12)

En ambos años se ve un pequeño crecimiento de incumplimiento de pagos al pasar los meses. En enero de 2018 hubieron mas incumplimientos que en diciembre de 2017. Es decir, cada vez se cumplen menos los pagos de las deudas al pasar el tiempo. Esto también se puede deber a que aumentaron los clientes. Habría que analizar la proporción de aumento de clientes con el aumento de los incumplimientos de pago.

In [365]:
df_reducido.drop(columns=['year'], inplace=True)

## b) Ingeniería de características

Eliminamos la variable "customer_ID" ya que no aporta información relevante que ayude a predecir el target

In [366]:
df_reducido.drop(columns=['customer_ID'], inplace=True)
df_reducido.head()

### Analizamos la cantidad de registros vacios

Para llevar a cabo el análisis de la cantidad de registros vacíos que posee cada columna utilizamos el porcentaje de los mismos, calculando la cantidad de registros vacíos sobre la cantidad total de registros que posee la columna. Decidimos que si dicho porcentaje es mayor a un 60% se eliminara dicha columna dada la falta de certeza sobre los datos.

In [367]:
def calculate_percentage_of_null_values(df: pd.DataFrame):
    number_of_rows = len(df)
    count_of_nulls: pd.Series = df.isnull().sum()
    percentages = []
    for index, count in count_of_nulls.iteritems():
        percentage = (count / number_of_rows) * 100
            
        percentages.append((index,percentage))
            
    return percentages

In [368]:
def eliminate_columns_with_high_percentage(percentages: list):
    columns_to_eliminate = []
    new_percentages = []
    for i in percentages:
        if(i[1] > 60):
            columns_to_eliminate.append(i[0])
        else:
            new_percentages.append(i)
        
    df_reducido.drop(columns=columns_to_eliminate , inplace = True)
    

    return new_percentages


In [369]:
percentages = calculate_percentage_of_null_values(df_reducido.drop(columns=['B_31','D_63', 'D_64', 'S_2','target']))
percentages = eliminate_columns_with_high_percentage(percentages)

Se optó por completar con el promedio de los datos a los registros de las columnas cuyos porcentajes de datos faltantes sea menor que el 60%.

Sin embargo, antes de realizar dicho procedimiento se buscan valores atípicos u outliers. Esto se debe a que en caso contrario de completar los datos faltantes con el promedio modificaría la varianza de cada columna, la cual es necesaria al utilizar Z-score en el análisis de valores atípicos. 


In [370]:
def columns_by_type(df: pd.DataFrame):
    columns = df.columns
    columns_by_type = [[],[],[],[],[]] # [D_columns ,S_columns,P_columns,B_columns,R_columns]
    
    
    for column in columns:
        if (column[0] == 'D'):
            columns_by_type[0].append(column)
        elif (column[0] == 'S'):
            columns_by_type[1].append(column)
        elif (column[0] == 'P'):
            columns_by_type[2].append(column)
        elif (column[0] == 'B'):
            columns_by_type[3].append(column)
        elif (column[0] == 'R'):
            columns_by_type[4].append(column)
    return columns_by_type

In [371]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import LinearRegression

columns = columns_by_type(df_reducido.drop(columns=['B_31','D_63', 'D_64', 'S_2','target']))

columns_S= columns[1]
df_columns = df_reducido[columns_S].copy()

In [372]:
lr = LinearRegression()
imp = IterativeImputer(estimator=lr,missing_values=np.nan, max_iter=20, random_state=10)
columns_without_nans = imp.fit_transform(df_columns)

### Detección de outliers

In [373]:
import scipy.stats as st

In [374]:
columns = df_reducido.drop(columns=['B_31','D_63', 'D_64', 'S_2','target']).columns
zscore_df = pd.DataFrame()
column_name = 'z_'
for column in columns:
      zscore_df[column_name + column] = st.zscore(df_reducido[column])

In [375]:
zscore_df.head()

In [376]:
outliers_df = pd.DataFrame()
for column in zscore_df.columns:
      if (sum(zscore_df[column].values > 3) > 0) or (sum(zscore_df[column].values < -3) > 0):
            outliers_df[column] = zscore_df[column]

In [ ]:
outliers_df.head(10)

In [ ]:
columns = df_reducido.drop(columns=['B_31','D_63', 'D_64', 'S_2','target']).columns
for column in columns:
    mean = np.mean(df_reducido[column])
    df_reducido[column].fillna(mean, inplace=True)

In [ ]:
df_reducido.drop(columns=['B_31','D_63', 'D_64', 'S_2','target']).isna().sum().sum()

### Transformación de variables

Se transforman las columnas categóricas a numéricas para poder trabajar con ellas con los modelos que las requieran

In [ ]:
dummies_cols = ['D_63', 'D_64']
df_reducido_dummies = pd.get_dummies(df_reducido, columns=dummies_cols)
df_reducido_dummies.head(10)